In [1]:
%%writefile workspace.json
{
"subscription_id" : "fe38c376-b42a-4741-9e7c-f5d7c31e5873",
"resource_group" : "XXXX",
"workspace_name" : "XXX"
}

Writing workspace.json


In [5]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace

ml_client = MLClient.from_config(credential=DefaultAzureCredential(), path="workspace.json")
ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7f691e5b70>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [10]:
import uuid

AOAI_endpoint = "https://openaiykus.openai.azure.com/" 
key_value="8a0342b4808245f6824e373cd2a55bb7"

keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="aoai-endpoint", value = AOAI_endpoint)
keyvault.set_secret(name="key",value=key_value)


In [11]:
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [12]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = aoai_endpoint, 
  api_key=aoai_key,  
  api_version="2023-05-15"
)

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
)

print(response.choices[0].message.content)

Yes, other Azure AI services, like Azure Cognitive Services, do support customer-managed keys as well. However, it's encouraged to check specific documentation or support channels for each individual service as capabilities and features can vary.
